In [1]:
import pandas as pd
import nltk

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import WordNetLemmatizer

# from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import pickle

import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset = pd.read_csv("../data/dataset/dataset.csv", sep=';', encoding="ISO-8859-1")

In [3]:
dataset

,Tweet,label
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",baterai negatif
1,Kmera iphone 11 mmeng beda,kamera positif
2,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera negatif
3,Apple meluncurkan iPhone 11 dengan peningkatan...,baterai positif
4,"xs max, senderrr. tapi rugi sih kalo kamu beli...",spesifikasi positif
...,...,...
743,ga keren banget masa charger bawaannya belum f...,baterai negatif
744,"yaelah belum fast charging, lama nih kalo ngec...",baterai negatif
745,butuh waku lama buat ngecharge kalo pake charg...,baterai negatif
746,udah mahal begini masa chargernya blm fast cha...,baterai negatif


## Clean Data

1. Menghilangkan simbol

In [4]:
def clean(data):
    data['Tweet_Parsed_1'] = data['Tweet'].str.replace("\\\\r", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("\\\\n", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("\n", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("\r", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("&amp", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("&gt", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("&lt", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("    ", " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace(r'''(?i)\b((?:https|http?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ")
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace('"', '')
    data['Tweet_Parsed_1'] = data['Tweet_Parsed_1'].str.replace("\\\\x[a-zA-z0-9][a-zA-z0-9]","")
    return data

In [5]:
dataset = clean(dataset)

2. Lowercase

In [6]:
def lowercase(data):
    data['Tweet_Parsed_2'] = data['Tweet_Parsed_1'].str.lower()
    return data

In [7]:
dataset = lowercase(dataset)

3. Menghilangkan tanda baca

In [8]:
def removepunct(data):
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_2']
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace("'s","")
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace("can't","can not")
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace("ain't","am not")
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace("n't"," not")
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace("'re"," are")
    data['Tweet_Parsed_3'] = data['Tweet_Parsed_3'].str.replace('[^a-zA-Z0-9]+',' ')
    return data

In [9]:
dataset = removepunct(dataset)

## Normalization

In [10]:
normal = pd.read_csv("../data/normalisasi/kamusnormalisasi.csv", sep=",", encoding='latin-1',header=None,names=["non-standar","standar"])

In [11]:
normal

,non-standar,standar
0,kmera,kamera
1,dijangka,dalam
2,dimuatkan,dimasukan
3,kapasiti,kapasitas
4,bateri,baterai
...,...,...
284,say,mengatakan
285,bangettt,sekali
286,guno,berguna
287,wowowoowow,wow


In [12]:
nonstandar = normal['non-standar'].values.tolist()
standar = normal['standar'].values.tolist()

In [13]:
def normalizeText(text, st_, nst_):
#     text = text.split(" ") 
    text = str(text).split(" ")
    for i in range(len(text)):
        if text[i] in nst_:
            index = nst_.index(text[i])
            text[i] = st_[index]
    return ' '.join(map(str, text))

def normalizeData(data, st_, nst_):
    data['Tweet_Parsed_4'] = data['Tweet_Parsed_3']
    data['Tweet_Parsed_4'] = data['Tweet_Parsed_4'].map(lambda com : normalizeText(com,st_,nst_))
    return data

In [14]:
dataset = normalizeData(dataset,standar,nonstandar)

In [15]:
dataset.to_csv("../data/dataset/data_normalizazionprocess_checkpoint.csv",index = False,sep=";")

1. Menghilangkan Angka

In [16]:
def removeNumb(data):
    data['Tweet_Parsed_4'] = data['Tweet_Parsed_4'].str.replace('[^a-zA-Z]+',' ')
    return data

In [17]:
dataset = removeNumb(dataset)

2. Menghilangkan kata tidak bermakna
Menggunakan stopword dari NLTK dan penlusuran dataset

In [18]:
meaningless = pd.read_csv("../data/stopword/stopword.csv", header=None, names=['stop'])
meaningless = meaningless['stop'].tolist()

In [19]:
meaningless

['stop',
 'leaks',
 'sis',
 'guys',
 'boi',
 'dah',
 'btw',
 'lol',
 'koook',
 'the',
 'way',
 'by',
 'pun',
 'bajingan',
 'sumpah',
 'yaampuuun',
 'siiih',
 'shopee',
 'shopeeid',
 'nih',
 'sih',
 'si',
 'ayo',
 'ya',
 'oh',
 'to',
 'yung',
 'hai',
 'lang',
 'nya',
 'haha',
 'ya',
 'nih',
 'shopeedarirumah',
 'tolong',
 'wkwk',
 'kah',
 'dong',
 'doang',
 'hai',
 'hihi',
 'hiks',
 'huft',
 'yang',
 'ya',
 'ni',
 'sialla',
 'nder',
 'ken',
 'ha',
 'mai',
 'amik',
 'sihh',
 'ko',
 'btsarmy',
 'btsfesta',
 'bts',
 'twt',
 'heu',
 'anjrit',
 'lh',
 'alang',
 'senderr',
 'ekekekeke',
 'sender',
 'atuh',
 'babi',
 'aja',
 'gitu',
 'duh',
 'bro',
 'huhu',
 'hehehe',
 'ahahhahah',
 'haih',
 'superb',
 'ngentot',
 'minn',
 'yah',
 'iphone',
 'pro',
 'max',
 'xs',
 'apple',
 'x',
 'promax',
 'allah',
 'senderrr',
 'ada',
 'adalah',
 'adanya',
 'adapun',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'anta

In [20]:
def removeMeaningless(data,meaningless_):
    for word in meaningless_:
        regex_meaningless = r"\b" + word + r"\b"
        data['Tweet_Parsed_4'] = data['Tweet_Parsed_4'].str.replace(regex_meaningless, '')
    return data

In [21]:
dataset = removeMeaningless(dataset,meaningless)

In [22]:
def clean_space(text):
    text_split = text.split(' ')
    text_clean = [i for i in text_split if i != '']
    text_fix = ' '.join(map(str,text_clean))
    return text_fix

def clean_space_data(data):
    data['Tweet_Parsed_4'] = data['Tweet_Parsed_4']
    data['Tweet_Parsed_4'] = data['Tweet_Parsed_4'].map(lambda com : clean_space(com))
    return data

In [23]:
dataset = clean_space_data(dataset)

In [24]:
dataset['Tweet'].loc[4]

'xs max, senderrr. tapi rugi sih kalo kamu beli xs max baru sekarang.. i mean langsung ke iphone 11 juga gapapa sebenernya secara speknya lebih bagus terussss harganya gak jauh beda.'

In [25]:
dataset['Tweet_Parsed_1'].loc[4]

'xs max, senderrr. tapi rugi sih kalo kamu beli xs max baru sekarang.. i mean langsung ke iphone 11 juga gapapa sebenernya secara speknya lebih bagus terussss harganya gak jauh beda.'

In [26]:
dataset['Tweet_Parsed_2'].loc[4]

'xs max, senderrr. tapi rugi sih kalo kamu beli xs max baru sekarang.. i mean langsung ke iphone 11 juga gapapa sebenernya secara speknya lebih bagus terussss harganya gak jauh beda.'

In [27]:
dataset['Tweet_Parsed_3'].loc[4]

'xs max senderrr tapi rugi sih kalo kamu beli xs max baru sekarang i mean langsung ke iphone 11 juga gapapa sebenernya secara speknya lebih bagus terussss harganya gak jauh beda '

In [28]:
dataset['Tweet_Parsed_4'].loc[4]

'rugi beli maksud langsung tidak speknya lebih bagus harganya tidak beda'

In [29]:
def fixData(data, stopword=True):
    if stopword == True:
        data_fix = data.drop(['Tweet_Parsed_1', 'Tweet_Parsed_2','Tweet_Parsed_3'],axis=1)
        data_fix = data_fix.rename(columns={'Tweet_Parsed_4': 'Tweet_Parsed'})
    else:
        data_fix = data.drop(['Tweet_Parsed_1', 'Tweet_Parsed_2','Tweet_Parsed_4'],axis=1)
        data_fix = data_fix.rename(columns={'Tweet_Parsed_3': 'Tweet_Parsed'})
    return data_fix

In [30]:
dataset = fixData(dataset, stopword=True)

In [31]:
def handleEmptyString(data):
    data.loc[data['Tweet_Parsed'] == '', ['Tweet_Parsed']] = " "
    return data

In [32]:
dataset = handleEmptyString(dataset)

In [33]:
dataset.to_csv("../data/dataset/datasetpraproses.csv",index = False,sep=";")